<a href="https://colab.research.google.com/github/elabeeel/News-classification/blob/main/clasificacion_noticias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import nltk
nltk.download('stopwords')

# Ruta al archivo CSV en tu Google Drive
business = '/content/drive/MyDrive/clasificacion noticias/business_data.csv'
education = '/content/drive/MyDrive/clasificacion noticias/education_data.csv'
entertainment = '/content/drive/MyDrive/clasificacion noticias/entertainment_data.csv'
sports = '/content/drive/MyDrive/clasificacion noticias/sports_data.csv'
technology = '/content/drive/MyDrive/clasificacion noticias/technology_data.csv'
# Cargar los archivos CSV
df1 = pd.read_csv(business)
df2 = pd.read_csv(education)
df3 = pd.read_csv(entertainment)
df4 = pd.read_csv(sports)
df5 = pd.read_csv(technology)

# Combinar DataFrames
dfs = [df1, df2, df3, df4, df5]
df = pd.concat(dfs)

# Mezclar datos
df = shuffle(df)

# Eliminar columnas innecesarias
df.drop(columns=["url"], inplace=True)

# Mapear categorías a valores numéricos
def map_category(category):
    if category == "business":
        return 0
    elif category == "education":
        return 1
    elif category == "entertainment":
        return 2
    elif category == "sports":
        return 3
    elif category == "technology":
        return 4
    else:
        return -1  # Categoría desconocida

df['Category'] = df['category'].apply(map_category)
df.drop(['category'], axis=1, inplace=True)

# Combinar columnas de texto
df['Content'] = df['headlines'] + ' ' + df['description'] + ' ' + df['content']
df.drop(['headlines', 'description', 'content'], axis=1, inplace=True)

# Stemming y eliminación de stopwords
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()

def stemming(content):
    content = re.sub('[^a-zA-Z]', ' ', content)
    content = content.lower()
    content = content.split()
    content = [ps.stem(word) for word in content if not word in stop_words]

    content = ' '.join(content)
    return content

df['Content'] = df['Content'].apply(stemming)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
# Dividir datos en conjuntos de entrenamiento y prueba
X = df['Content']
Y = df['Category']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=42)

# Vectorizar datos con TF-IDF
vc = TfidfVectorizer()
X_train = vc.fit_transform(X_train)
X_test = vc.transform(X_test)

# Entrenar modelo SVM
model = SVC()
model.fit(X_train, Y_train)

# Evaluar el modelo
Y_pred = model.predict(X_test)
print("Precisión del modelo:", accuracy_score(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))

Precisión del modelo: 0.9875
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       400
           1       0.99      0.98      0.99       400
           2       1.00      1.00      1.00       400
           3       0.99      1.00      1.00       400
           4       0.97      0.97      0.97       400

    accuracy                           0.99      2000
   macro avg       0.99      0.99      0.99      2000
weighted avg       0.99      0.99      0.99      2000



In [9]:
def make_predictions(headlines, description, content):
    # Preprocesar el texto de la noticia
    text = headlines + ' ' + description + ' ' + content
    text = stemming(text)
    text = vc.transform([text])  # Vectorizar el texto

    # Predecir la categoría
    val = model.predict(text)

    # Convertir la predicción numérica a categoría
    def val_to_category(val):
        if val == 0:
            return "business"
        elif val == 1:
            return "education"
        elif val == 2:
            return "entertainment"
        elif val == 3:
            return "sports"
        elif val == 4:
            return "technology"
        else:
            return "desconocida"

    category = val_to_category(int(val[0]))
    print("Según el texto, esta noticia es sobre:", category)

In [10]:
# Ejemplo de noticia de deportes
headlines = "Equipo A gana el campeonato"
description = "El equipo A venció al equipo B en la final del campeonato."
content = "El partido fue muy emocionante y se definió en el último minuto..."
make_predictions(headlines, description, content)

# Ejemplo de noticia de tecnología
headlines = "Nueva versión del sistema operativo"
description = "La compañía X lanza una nueva versión de su sistema operativo con mejoras..."
content = "Las nuevas características incluyen..."
make_predictions(headlines, description, content)

Según el texto, esta noticia es sobre: sports
Según el texto, esta noticia es sobre: technology
